In [ ]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import ast
import math

nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocess_user_input(user_input):

    words = user_input.split()
    lemmatizedWords = []
    for word in words:
        word = word.lower()

        lemword = lemmatizer.lemmatize(word)

        lemmatizedWords.append(lemword)

    final_output = ' '.join(lemmatizedWords)

    return final_output


In [ ]:
# Cosine similarity function
def cosine_similarity(v1, v2):

    dotProduct = 0
    for x,y in zip(v1,v2):
        dotProduct += x*y

    normv1 = 0
    for x in v1:
        normv1 += x ** 2
    normv1 = normv1 ** 0.5

    normv2 = 0
    for x in v2:
        normv2 += x ** 2
    normv2 = normv2 ** 0.5

    if normv1 == 0 or normv2 == 0:
        return 0.0
    similarity = dotProduct / (normv1 * normv2)

    return similarity

In [ ]:
def parse_ingredients(ingredients_list):
    # Remove any extra characters such as quotes or spaces around the list
    cleaned_ingredients = ingredients_list.strip().strip("'").strip("[]")
    # Split by commas and remove any extra spaces
    ingredients = cleaned_ingredients.split("', '")
    final = " ".join(ingredients)
    print(final)
    return final

In [ ]:
def preprocess_data(filepath):
    # Read file manually without pandas
    with open(filepath, 'r') as file:
        lines = file.readlines()

    header = lines[0].strip().split(',')
    data = []

    for line in lines[1:]:
        row = line.strip().split(',')
        # Handle cases where the row has fewer elements than the header
        row_data = {header[i]: row[i] if i < len(row) else '' for i in range(len(header))}
        data.append(row_data)

    # Extract required columns and process ingredients
    for row in data:
        # Clean and parse the ingredients list
        row['ingredients_text'] = parse_ingredients(row['ingredients_list'])

    # Create a vocabulary for TF-IDF and compute TF-IDF values manually
    vocabulary = {}
    doc_count = {}
    tfidf_matrix = []

    # Build vocabulary and term frequencies
    for row in data:
        words = row['ingredients_text'].split()
        row_tfidf = {}
        for word in words:
            if word not in vocabulary:
                vocabulary[word] = len(vocabulary)
                doc_count[word] = 0
            doc_count[word] += 1
            row_tfidf[word] = row_tfidf.get(word, 0) + 1
        tfidf_matrix.append(row_tfidf)

    # Compute TF-IDF for each word
    total_docs = len(data)
    dense_tfidf_matrix = []
    for row_tfidf in tfidf_matrix:
        dense_row = [0] * len(vocabulary)
        for word, count in row_tfidf.items():
            tf = count
            idf = math.log(total_docs / (1 + doc_count[word]))  # Applying smoothing
            dense_row[vocabulary[word]] = tf * idf
        dense_tfidf_matrix.append(dense_row)

    return data, vocabulary, dense_tfidf_matrix


In [ ]:
# Main execution
file = "/content/drive/MyDrive/Colab Notebooks/recipes.csv"
data, vocabulary, dense_tfidf_matrix = preprocess_data(file)

In [ ]:
# Get recipe recommendations based on cosine similarity
def get_recommendations(user_input, vocabulary, dense_tfidf_matrix, data):
    user_input = preprocess_user_input(user_input)

    # Vectorize user input
    user_vector = [0] * len(vocabulary)
    inputWords = user_input.split()
    for word in inputWords:
        if word in vocabulary:
            user_vector[vocabulary[word]] += 1

    # Compute similarity with each recipe
    similarity_scores = []
    for recipe_vector in dense_tfidf_matrix:
        similarity_scores.append(cosine_similarity(user_vector, recipe_vector))

    # Get top 5 recipes
    top_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:5]
    for index in top_indices:
        recipe = data[index]
        print("Recipe Name:", recipe['recipe_name'])
        print("Ingredients:", recipe['ingredients_text'])
        print("Ratings:", recipe['aver_rate'])
        print("-" * 50)


In [ ]:
def kmeans_clustering_optimized(data, k, max_iterations=10, tolerance=1e-4):
    # Convert data to a numpy array for vectorized operations
    data = np.array(data)
    n_samples, n_features = data.shape

    # Initialize centroids using K-Means++ for better convergence
    centroids = [data[np.random.randint(n_samples)]]
    for _ in range(1, k):
        distances = np.min([np.linalg.norm(data - centroid, axis=1)**2 for centroid in centroids], axis=0)
        probabilities = distances / np.sum(distances)
        cumulative_probabilities = np.cumsum(probabilities)
        r = np.random.rand()

        for j, p in enumerate(cumulative_probabilities):
            if r < p:
                centroids.append(data[j])
                break

    centroids = np.array(centroids)

    # Iterate to refine centroids
    for _ in range(max_iterations):
        # Compute cosine similarities in a vectorized manner
        dot_products = np.dot(data, centroids.T)
        data_norms = np.linalg.norm(data, axis=1, keepdims=True)
        centroid_norms = np.linalg.norm(centroids, axis=1, keepdims=True).T
        similarities = dot_products / (data_norms @ centroid_norms + 1e-8)

        # Assign clusters based on maximum similarity
        cluster_labels = np.argmax(similarities, axis=1)

        # Update centroids
        new_centroids = np.array([
            np.mean(data[cluster_labels == c], axis=0) if len(data[cluster_labels == c]) > 0 else centroids[c]
            for c in range(k)
        ])

        # Check for convergence
        if np.linalg.norm(new_centroids - centroids) < tolerance:
            break
        centroids = new_centroids

    # Group data points into clusters
    clusters = []
    for c in range(k):
        cluster_data = data[cluster_labels == c]
        clusters.append(cluster_data)
    return centroids, clusters


In [ ]:
centroids, clusters = kmeans_clustering_optimized(dense_tfidf_matrix, k=3)
print("Clustering completed with centroids:\n")


Clustering completed with centroids:



In [ ]:
#user_input = input("Enter ingredients: ").strip()
user_input = "chicken"
get_recommendations(user_input, vocabulary, dense_tfidf_matrix, data)


Recipe Name: Chicken Parmesan Burger
Ingredients: "['ground chicken
Ratings: 4
--------------------------------------------------
Recipe Name: Cheesy Chicken Meatballs
Ingredients: "['ground chicken
Ratings: 4.39
--------------------------------------------------
Recipe Name: Chicken Parmesan Meatball Sliders
Ingredients: "['ground chicken
Ratings: 4.43
--------------------------------------------------
Recipe Name: Chicken Lettuce Wraps
Ingredients: "['ground chicken
Ratings: 3.79
--------------------------------------------------
Recipe Name: Chicken Cordon Bleu Bites
Ingredients: "['ground chicken
Ratings: 4.02
--------------------------------------------------
